In [1]:
import findspark
findspark.init()

In [2]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import HiveContext, Row
conf = SparkConf().setAppName("chihweihsu").setMaster("local[*]")
sc = SparkContext(conf=conf)
sqlContext = HiveContext(sc)

In [ ]:
from pyspark.sql import Row
data = sc.parallelize([("18679196770",0,1,45.3,4,"3:12"),\
                       ("18679196771",3,2,45.3,14,"5:18"),\
                       ("18679196772",5,0,45.3,34,"3:22"),\
                       ("18679196773",6,4,45.3,44,"7:12"),\
                       ("18679196774",8,8,45.3,54,"0:11"),\
                       ("18679196775",9,9,45.3,64,"1:13"),\
                       ("18679196776",2,2,45.3,74,"2:19"),\
                       ("18679196777",1,5,45.3,84,"6:27"),\
                       ("18679196778",5,3,45.3,94,"2:18")]).map(lambda x: (x[0], x[1:]))
print data.collect()
wp = data.partitionBy(8,lambda k: int(k)%8)
print wp.collect()
print wp.map(lambda t: t[0]).collect()
print wp.glom().collect()

In [3]:
# data1 = sc.parallelize("18679196770|0,1,1,0,0,1,0,0,0|1_10,2_9,3_6,0_18,0_7,0_9,1_19,0_8,1_9") # F1,F5
# data2 = sc.parallelize("18679196770|12.3|24") # F3,F4
# data3 = sc.parallelize(["18679196770|34"]) # F2
# print data3.collect()
# data3.alias('a').join(df2.alias('b'),col('b.id') == col('a.id')).select([col('a.'+xx) for xx in a.columns] + [col('b.other1'),col('b.other2')])
# data.map()
a = sc.parallelize(["18679196770|0,0,0,0,0,0,0,0,0|1_10,2_9,3_6,0_18,0_7,0_9,1_19,0_8,1_9", \
                    "18679196771|0,1,1,0,None,1,1,0,0|1_10,2_9,3_6,0_18,0_7,0_9,1_19,0_8,1_9", \
                    "18679196772|0,1,0,0,0,1,1,0,0|1_10,2_9,3_6,0_18,0_7,0_9,1_19,0_8,1_9", \
                    "18679196773|0,1,1,0,0,0,0,0,0|1_10,2_9,3_6,0_18,0_7,0_9,1_19,0_8,1_9" \
                   ])
a.saveAsTextFile('./hdfs/dumiao/callLog')
print a.collect()
b = sc.parallelize(["18679196770|12.3|None","18679196771|230|24", "18679196772|None|4"])
b.saveAsTextFile('./hdfs/dumiao/userInfo')
print b.collect()
c = sc.parallelize(["18679196770|None","18679196772|1", "18679196773|15"])
c.saveAsTextFile('./hdfs/dumiao/cuishou')
print c.collect()
# b = sc.parallelize([['p1', 'a'], ['p2', 'b'], ['p3', 'c']]).toDF(["other", "b_id"])

# c = a.join(b, a.tel == b.tel).drop(b.tel)

['18679196770|0,0,0,0,0,0,0,0,0|1_10,2_9,3_6,0_18,0_7,0_9,1_19,0_8,1_9', '18679196771|0,1,1,0,None,1,1,0,0|1_10,2_9,3_6,0_18,0_7,0_9,1_19,0_8,1_9', '18679196772|0,1,0,0,0,1,1,0,0|1_10,2_9,3_6,0_18,0_7,0_9,1_19,0_8,1_9', '18679196773|0,1,1,0,0,0,0,0,0|1_10,2_9,3_6,0_18,0_7,0_9,1_19,0_8,1_9']
['18679196770|12.3|None', '18679196771|230|24', '18679196772|None|4']
['18679196770|None', '18679196772|1', '18679196773|15']


In [14]:
import itertools

def deal_call(item):
    item = item.split('|')
    max_blank = max([len(list(v)) for k,v in itertools.groupby(item[1].split(',')) if k == '0'])
    tmp = [ int(j)  for i in item[2].split(',') for j in i.split('_')]
    call_pct = "{:.2f}".format(sum(tmp[::2])*1.0/sum(tmp[1::2]))
    return (item[0], (max_blank, call_pct))

def deal_info(item):
    tmp = item.split('|')
    return (tmp[0],(tmp[1] if tmp[1] != 'None' else None,tmp[2] if tmp[2] != 'None' else None))

def deal_cuishou(item):
    tmp = item.split("|")
    return (tmp[0], tmp[1] if tmp[1] != 'None' else None)

df_15 = sc.textFile("./hdfs/dumiao/callLog").map(deal_call).persist()
df_34 = sc.textFile("./hdfs/dumiao/userInfo").map(deal_info).persist()
df_2 = sc.textFile("./hdfs/dumiao/cuishou").map(deal_cuishou).persist()
bc_cuishou = sc.broadcast(df_2.collectAsMap())
# print df_15.collect(), df_34.collect(), df_2.collectAsMap()

In [24]:

def deal_all(item):
    """
    (u'18679196772', ((3, '0.08'), (u'12.3', u'24')))
    (u'18679196773', ((3, '0.08'), None))
    """
    tel = item[0]
    max_blank = item[1][0][0] if item[1][0] and item[1][0][0] else -8887
    call_pct = item[1][0][1] if item[1][0] and item[1][0][1] else -8887
    bill_fee = item[1][1][0] if item[1][1] and item[1][1][0]  else -8887
    innet_months = item[1][1][1] if item[1][1] and item[1][1][1] else -8887
    cuishou_nums =  bc_cuishou.value.get(tel, -8887)
    cuishou_nums = cuishou_nums if cuishou_nums else -8887
#     return (tel,max_blank, cuishou_nums, bill_fee, innet_months, call_pct)
    return (tel, (max_blank, float(cuishou_nums), float(bill_fee), float(innet_months), float(call_pct)))
df = df_15.fullOuterJoin(df_34).map(deal_all).persist()
print df.collect()

[(u'18679196772', (3, 1.0, -8887.0, 4.0, 0.08)), (u'18679196773', (6, 15.0, -8887.0, -8887.0, 0.08)), (u'18679196770', (9, -8887.0, 12.3, -8887.0, 0.08)), (u'18679196771', (2, -8887.0, 230.0, 24.0, 0.08))]


In [25]:
import bisect
RATING_RULES = [
    ([-8887, 1, 2, 7], [-17, 25, 17, -25, -50]),  # hd_maxofflinedaysin3months
    ([1], [23, -53]),  # dhb_callTelTotalNumCollHist
    ([-8887, 30, 55, 118], [-25, -8, 15, 30, 88]),  # dinner_rent_fee
    ([-8887, 15, 25, 40, 52, 72], [-9, -69, -25, -17, -1, 31, 42]),  # innet_months
    ([-8887, 0.01, 0.02, 0.03, 0.05], [-18, 27, 11, -21, -24, -63]),  # hd_contactearlymorningrate
]

def cul(item):
    score_total = 603
    for n,i in enumerate(RATING_RULES):
        idx = bisect.bisect_left(i[0], item[1][n])
        score_total += i[1][idx]
    return (item[0], score_total)

df.map(cul).collect()

[(u'18679196772', 444),
 (u'18679196773', 428),
 (u'18679196770', 496),
 (u'18679196771', 643)]

In [ ]:
import itertools
import time

t = time.time()
a = "18679196770|0,1,1,0,0,1,0,0,0|1_10,2_9,3_6,0_18,0_7,0_9,1_19,0_8,1_9"
a = a.split('|')
b = max([len(list(v)) for k,v in itertools.groupby(a[1].split(',')) if k == '0'])
c = [ int(j)  for i in a[2].split(',') for j in i.split('_')]
c = "{:.2f}".format(sum(c[::2])*1.0/sum(c[1::2]))
print time.time()-t
print a[0],b,c


In [ ]:
print [[y for y in range(x, x+3)] for x in [1, 4, 7]]

In [ ]:
import pandas as pd
left = pd.DataFrame({'key': ['foo', 'bar'], 'val': [1, 2]}).set_index('key')
right = pd.DataFrame({'key': ['foo', 'bar'], 'val': [4, 5]}).set_index('key')
print left, right
left.join(right, lsuffix='_l', rsuffix='_r')

In [ ]:
# 保存数据
# wp.saveAsTextFile('./hdfs/dumiao/') # rdd
# wp.toDF().write.save('/target/path/', format='parquet', mode='append') # dataframe
wp.saveAsPickleFile('./hdfs/dumiao/1', 3)


In [ ]:
# 读取数据
# df = sc.textFile("./hdfs/dumiao/1")
df = sc.pickleFile("./hdfs/dumiao/1", 5)
# print df
print df.collect()

df.map(lambda x: (x[0],x[1][1:])).collect()


In [ ]:
import random
import time
import itertools

random.seed(0)
a = [ '1' if random.random()>0.4 else ' ' for i in range(100)]
print a, ''.join(a)
print [len(x) for x in ''.join(a).split()]
t = time.time()
print max([len(x) for x in ''.join(a).split()])
print time.time()-t

t = time.time()
print max([len(list(v)) for k,v in itertools.groupby(a)])
print time.time()-t


In [ ]:
a = '22'
print a or None

In [ ]:
import time,math,bisect
RATING_RULES = [
    ([-8887, 1, 2, 7], [-17, 25, 17, -25, -50]),  # hd_maxofflinedaysin3months
    ([1], [23, -53]),  # dhb_callTelTotalNumCollHist
    ([-8887, 30, 55, 118], [-25, -8, 15, 30, 88]),  # dinner_rent_fee
    ([-8887, 15, 25, 40, 52, 72], [-9, -69, -25, -17, -1, 31, 42]),  # innet_months
    ([-8887, 0.01, 0.02, 0.03, 0.05], [-18, 27, 11, -21, -24, -63]),  # hd_contactearlymorningrate
]

def cul(item):
    score_total = 603
    for n,i in enumerate(RATING_RULES):
        idx = bisect.bisect_left(i[0], item[1][n])
        score_total += i[1][idx]
    return (item[0], score_total)

cul((u'18679196772', (3.0, 24.0, 12.3, 24.0, 0.08)))

In [ ]:
bisect.bisect_left([1], -0)


In [11]:
a = 0
b = a if a else None
print b

None


In [26]:
"03504ABC4C12A13081753F891D8BC6AC841EDE06|0,0,".split("|")

['03504ABC4C12A13081753F891D8BC6AC841EDE06', '0,0,']

In [29]:
import os
os.path.basename('./dumiao/info')
os.path.dirname('./dumiao/info')

'./dumiao'

In [2]:
import datetime
from datetime import datetime as DT
def cut_term(term, cut_day=1, format_str="%Y-%m-%d"):
        """
        计算周期获取，默认7天
        """
        return [(DT.strptime(term, format_str) - datetime.timedelta(days=i)).strftime(format_str) for i in xrange(1, cut_day+1)]

print cut_term('2018-10-09')

['2018-10-08']
